# Problem Set 6 (Total points: 100)


### Q1. Markov Decision Process [20 points]
Imagine an MDP corresponding to playing slot machines in a casino. Suppose you start with \$20\$ cash to spend in the casino,  and you decide to play until you lose all your money or until you double your money (i.e., increase your cash to at least \$40\$). There are two slot machines you can choose to play: 1) slot machine X costs \$10\$ to play and will pay out \$20\$ with probability \$0.05\$ and will pay \$0\$ otherwise;
and \$2)\$ slot machine Y costs \$20\$ to play and will pay out \$30\$ with probability 0.01 and \$0 \$ otherwise. As you are playing, you keep choosing machine X or Y at each turn.

Write down the MDP that corresponds to the above problem. Clearly specify the state space, action space, rewards and transition probabilities. Indicate which state(s) are terminal. Assume that the discount factor γ = 1.

**Notes:** There are several valid ways to specify the MDP, so you have some flexibility in your solution. For example, rewards can take many different forms, but overall you should get a higher reward for stopping when you double your money than when you lose all your money!

States $S_i$ where $i \in \{0, 10, 20, 30, 40\}$ and $s_0$ and $s_{40}$ are terminal

Actions $A = \{X, Y\}$ denotes the slot machine we decide to play with.

Rewards: $R(s_i, a, s_j) = j - i $

Rewards: $R(s_i, a, s_0) = 1 $ if $s_0 == s_{40}$, else 0, would also work. 

Transition probabilities(there are several valid ways to define these):

$P(s_{i-10}|s_i, X) = 0.95$

$P(s_{i+10}|s_i, X) = 0.05$

$P(s_{i-20}|s_i, Y) = 0.99$

$P(s_{i+10}|s_i, Y) = 0.01$

The above transitions are only valid for $X$ in states $s_{10}, s_{20}, s_{30}$, and for $Y$ in states $s_{20}, s_{30}$. All other transitions do not exist(have probability 0)

## Reinforcement Learning
In the remainder of the problem set you will implement the Q-Learning Algorithm to solve the "Frozen Lake" problem.
We​ ​will​ ​use​ ​OpenAI’s​ ​gym​ ​package​ ​to​ ​develop​ ​our​ ​solution​ ​in​ ​Python.​ 

### OpenAI Gym Setup 
​Read the​ ​set-up​ ​instructions for​ ​Gym​  [here](https://gym.openai.com/docs/).​ ​The​ ​instructions​ ​also​ ​give​ ​a​ ​good​ ​overview​ ​of​ ​the​ ​API​ ​for​ ​this​ ​package,​ ​so​ ​please​ ​read through​ ​it​ ​before​ ​proceeding.​



### Frozen Lake
Instead​ ​of​ ​using​ ​CartPole,​ ​we’re​ ​going​ ​to​ ​be​ ​using​ ​the​ [​​FrozenLake](https://gym.openai.com/envs/FrozenLake-v0/) environment. Read through the code for this environment by following the Github link.​ <br>

Winter​ ​is​ ​quickly​ ​approaching,​ ​and​ ​we​ ​have​ ​to​ ​worry​ ​about​ ​navigating​ ​frozen​ ​lakes.​ ​It’s​ ​only early November,​ ​
so​ ​the​ ​lakes​ ​haven’t​ ​completely​ ​frozen​ ​and​ ​if​ ​you​ ​make​ ​the​ ​wrong​ ​step​ ​you​ ​may​ ​fall​ ​through. 
We’ll​ ​need​ ​to​ ​learn​ ​how​ ​to​ ​get​ ​to​ ​our​ ​destination​ ​when​ ​stuck​ ​on​ ​the​ ​ice,​ ​without​ ​falling​ ​in.
The​ ​lake​ ​we’re​ ​going​ ​to​ ​consider​ ​is a​ ​square​ ​lake​ ​with​ ​spots​ ​to​ ​step​ ​on​ ​in​ ​the​ ​shape​ ​of​ ​a​ ​grid.​ ​​<br>

The surface is described using a 4x4 grid like the following

        S F F F 
        F H F H
        F F F H
        H F F G

​Each​ ​spot​ ​can have​ ​one​ ​of​ ​four​ ​states:
- S:​ ​starting​ ​point.
- G:​ ​goal​ ​point.
- F:​ ​frozen​ ​spot,​ ​where​ ​it’s​ ​safe​ ​to​ ​walk.
- H:​ ​hole​ ​in​ ​the​ ​ice,​ ​where​ ​it’s​ ​not​ ​safe​ ​to​ ​walk.<br>


For example, consider the lake, <br>
![alt text](https://cs-people.bu.edu/sunxm/frozen_lake_example.png)

There are four possible actions: UP, DOWN, LEFT, RIGHT. Although we​ ​can​ ​see​ ​the​ ​path​ ​we​ ​need​ ​to​ ​walk,​ the agent does not. ​We’re​ ​going​ ​to​ ​train​ ​an​ ​agent​ ​to​ ​discover​ ​this​ ​via​ ​problem solving.​ ​However,​ ​walking​ ​on​ ​ice​ ​isn’t​ ​so​ ​easy!​ ​Sometimes​ ​you​ ​slip​ ​and​ ​aren’t​ ​able​ ​to​ ​take​ ​the​ ​step​ ​you intended.

The episode ends when you reach the goal or fall in a hole.

You receive a reward of 1 if you reach the goal, and zero otherwise.

#### Q2. Walking on the Frozen Lake [10 points]

Write a script that sets up the Frozen Lake environment and takes 10 walks through it, consisting of a maximum 10 randomly sampled actions during each walk. After each step, render the current state, and print out the reward and whether the walk is "done", i.e. in the terminal state, because the agent fell into a hole (stop if it is). In your own words, explain how this environment behaves. 

The environment behaves in a stochastic way, i.e. taking an action does not deterministically transition the agent into the next state. Instead the agent ends up in one of serveral states with some probability. The analogy here is that when you try to walk right on ice, you may slip and end up walking forward, etc.

In [1]:
!pip install gym

import gym,sys,numpy as np
from gym.envs.registration import register

In [2]:
max_step = 10
np.random.seed(42)

env = gym.make('FrozenLake-v0').unwrapped

# do a random walk
for i in range(max_step):
    env.reset()
    random_action = env.action_space.sample()
    state_new, reward, done, info = env.step(random_action)
    print('step', i + 1, ':',['Left', 'Down', 'Right', 'Up'][random_action])
    env.render()
    print('current state:', state_new)
    print('reward', reward)
    print('done?', done)
    if done: break

step 1 : Up
  (Up)
SFFF
FHFH
FFFH
HFFG
current state: 0
reward 0.0
done? False
step 2 : Down
  (Down)
SFFF
FHFH
FFFH
HFFG
current state: 4
reward 0.0
done? False
step 3 : Right
  (Right)
SFFF
FHFH
FFFH
HFFG
current state: 0
reward 0.0
done? False
step 4 : Down
  (Down)
SFFF
FHFH
FFFH
HFFG
current state: 1
reward 0.0
done? False
step 5 : Down
  (Down)
SFFF
FHFH
FFFH
HFFG
current state: 1
reward 0.0
done? False
step 6 : Up
  (Up)
SFFF
FHFH
FFFH
HFFG
current state: 1
reward 0.0
done? False
step 7 : Up
  (Up)
SFFF
FHFH
FFFH
HFFG
current state: 0
reward 0.0
done? False
step 8 : Down
  (Down)
SFFF
FHFH
FFFH
HFFG
current state: 4
reward 0.0
done? False
step 9 : Down
  (Down)
SFFF
FHFH
FFFH
HFFG
current state: 1
reward 0.0
done? False
step 10 : Right
  (Right)
SFFF
FHFH
FFFH
HFFG
current state: 1
reward 0.0
done? False


#### Q3. Q-Learning [50 points]

You will ​implement​ ​Q-learning to solve the problem.​ Assume that the environment has states S and actions A. Use the ​function​ ​signature​ ​provided​ below:

The pseudocode for Q-Learning was described in lecture, but for your reference, we provide it here:
![alt text](https://cs-people.bu.edu/sunxm/q-learning.png)


In [3]:
def q_learning(env, alpha=0.5, gamma=0.95, epsilon=0.1, num_episodes=500, RAND = False):
       
    """ Performs Q-learning for the given environment.
    Initialize Q to all zeros
    env: Unwrapped OpenAI gym environment.
    alpha: Learning rate parameter.
    gamma: Decay rate (future reward discount) parameter.
    num_episodes: Number of episodes to use for learning. 
    return: Q table, i.e. a table with the Q value for every <S, A> pair.
    """ 
    np.random.seed(42)
    state_num = env.nS
    action_num = env.nA
    Q = np.zeros((state_num, action_num))
    
    if RAND == True:
        Q = np.random.rand(state_num, action_num)
        terminal_state = [5, 7, 11, 12, 15]
        Q[terminal_state, :] = 0
    
    for epis in range(num_episodes):                       # loop through episodes
        
        cur_state = env.reset()
        done = False
        
        while not done:
            if np.random.rand() < 1 - epsilon:
                action = np.argmax(Q[cur_state, :])
            else:
                action = np.random.randint(0, action_num)
            next_state, reward, done, _ = env.step(action)  
            Q[cur_state, action] += alpha*(reward + gamma*np.argmax(Q[next_state,:]) - Q[cur_state, action])
            cur_state = next_state
            
    return Q
    

#### Q4. Main Function [20 points]
You also need to implement the main function to solve the entire FrozenLake-v0 problem, including setting up the Gym environment, 
calling the q-learning function as defined above, printing out the returned Q-table, etc. <br>

You should use the $\epsilon$-greedy algorithm (solving Exploration-Exploitation Dilemma) to generate actions for each state. Try `num_episodes` with different values, e.g. `num_episodes=500, 1000, 5000, 10000, 50000`. You should also try different initializations for the Q-table, i.e. Random Initialization and Zero Initialization. 

Please do not change any default value of environment setting, such as `is_slippery`, if not mentioned above.

Provide​ ​the​ ​final​ ​Q-table​ ​of​ ​​FrozenLake-v0​ for **each <num_episode, init_method>** you have tried [10 points],​ ​and analyze ​what​ you observe [10 points]. 

If you are interested in testing your learned Q-learned, you can compute the win rate (the probability of agents to goal following the Q-table). Note that you should use `argmax` instead of epsilon greedy to choose the action at the current state. 
Win Rate is a good way to test whether you write your algorithm correctly.

Just for your reference, we test our implementation for 10000 trials after 50000-episode training using zero initialization and the win rate is 0.4985. You should achieve similar value under the same test.



In [ ]:
def winrate(Q, env, num_episodes):
    num_win = 0.
    
    for epis in range(num_episodes):
        
        cur_state = env.reset()
        done = False
        while not done:
            
            action = np.argmax(Q[cur_state, :])
            next_state, reward, done, _ = env.step(action)
            num_win += reward
            cur_state = next_state
    
    return num_win/num_episodes

def main():
    env = gym.make('FrozenLake-v0').unwrapped
    np.random.seed(42)

    print('output of Q-learning')
    Q = q_learning(env, num_episodes = 500, RAND = True)
    print('Q: ', Q)
    print(winrate(Q, env, num_episodes = 500))
        
if __name__ == '__main__':
    main()

output of Q-learning
Q:  [[2.3156105  2.75382185 2.69840237 2.85      ]
 [2.21593371 2.83687404 2.1554081  2.85      ]
 [2.1100158  2.66074217 2.67187359 2.85      ]
 [0.95620248 1.06664883 2.10340565 2.85      ]
 [0.48838856 0.52784219 0.95459414 0.19353756]
 [0.         0.         0.         0.        ]
 [1.81674138 0.02839981 0.01475556 0.16710326]
 [0.         0.         0.         0.        ]
 [0.48313145 0.65107202 0.03528444 0.83390001]
 [0.14484458 2.24216998 0.08075728 0.06787293]
 [0.86792496 0.52668685 0.01719426 0.02905188]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.11858592 0.48941271 1.1919105  0.11864013]
 [0.12404922 1.71171304 0.02261364 0.79066629]
 [0.         0.         0.         0.        ]]


In the situation of zero-initialization, when we only have 500 or even 5000 episodes, the agent never goes to the goal in these tries and then it gets no non-zero rewards during the whole traing. After trying more, like 50000 episodes, the agent gets to the final goal and it gets non-zero rewards from the environment and then it begins to learn. Thus, when the number of episodes grows up to 500000, Q matrix has non-zero entries.

In the situation of random-initialization, in the first 500 episodes, the agent gets to the final goal and starts learning, so the Q-values are non-zeros.

Random initialization will help the agent easily gets to the final goal in the early episodes.

#### Additional Instructions
If​ ​you’re​ ​new​ ​to​ OpenAI’s​ ​Gym,​ first ​go​ ​through​ ​OpenAI’s​ ​full​ ​tutorial listed​ ​earlier​ ​and​ ​visit​ ​the​ ​Appendix​ ​to​ ​this​ ​homework​ ​before​ ​proceeding.
Some additional rules:
- Only submit **original code** written entirely by you.
- **Permitted​​ non-standard ​​libraries**:​​ ​gym​,​​ ​numpy.
- **Only​ ​use​ ​numpy​ ​for​ ​random​ ​sampling,​ ​and​ ​seed​ ​at​ ​the​ ​beginning​ ​of​ ​each​ ​function​ ​with**:
np.random.seed(42)
- **Unwrap​ ​the​ ​OpenAI​ ​gym​ ​before​ ​providing​ ​them​ ​to​ ​these​ ​functions.** <br>
env​ ​=​ ​gym.make(“FrozenLake-v0”).unwrapped


## Appendix

This​ ​appendix​ ​includes​ ​references​ ​to​ ​APIs​ ​you​ ​may​ ​find​ ​useful.​ ​If​ ​the​ ​description​ ​sounds​ ​useful,​ ​check out
the​ ​respective​ ​package’s​ ​documentation​ ​for​ ​a​ ​much​ ​better​ ​description​ ​than​ ​we​ ​could​ ​provide.

#### Numpy
- np.zeros:​ ​N-dimensional​ ​tensor​ ​initialized​ ​to​ ​all​ ​0s.
- np.ones:​ ​N-dimensional​ ​tensor​ ​initialized​ ​to​ ​all​ ​1s.
- np.eye:​ ​N-dimensional​ ​tensor​ ​initialized​ ​to​ ​a​ ​diagonal​ ​matrix​ ​of​ ​1s.
- np.random.choice:​ ​Randomly​ ​sample​ ​from​ ​a​ ​list,​ ​allowing​ ​you​ ​to​ ​specify​ ​weights.
- np.argmax:​ ​Index​ ​of​ ​the​ ​maximum​ ​element.
- np.abs:​ ​Absolute​ ​value.
- np.mean:​ ​Average​ ​across​ ​dimensions.
- np.sum:​ ​Sum​ ​across​ ​dimensions.

### OpenAI Gym
- Environment​ ​(unwrapped):<br>
env.nS #​ ​Number​ ​of​ ​spaces. <br>
env.nA #​ ​Number​ ​of​ ​actions. <br>
env.P #​ ​Dynamics​ ​model.
